In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re

from konlpy.tag import *

In [2]:
from konlpy.tag import Mecab

m = Mecab(dicpath = 'C:/mecab/mecab-ko-dic')
m.nouns('A타입도 좋고, PD충전 되는 것도 좋아요. C타입 여러개 있어서 좋습니다')

['A타입', 'PD충전', '것', 'C타입', '개']

In [3]:
df = pd.read_parquet('C:/Users/songy/Desktop/Data/sentiment_data/여행용 어댑터_20240717 15_44_05.parquet')
df.head(3)

,brand_name,product_name,write_dt_lst,rating_lst,content_lst
0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행가서 잘 썼습니당~ 유럽용은 안써도 될듯 ㅠ
1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장난거 아니였으면 하네요
2,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장난거 아니였으면 하네요


In [4]:
len(df)

12418

In [5]:
df.drop_duplicates(inplace=True)
len(df)

12275

In [6]:
# hanspell 설치된 경로 
!pip show hanspell

Name: hanspell
Version: 0.1
Summary: editing hanspell package (for testing)
Home-page: https://github.com/Seokhyeon-Park/hanspell.git
Author: Seokbong
Author-email: dfbgq346q@gmail.com
License: 
Location: c:\users\songy\anaconda\envs\mecab\lib\site-packages
Requires: requests
Required-by: 


In [7]:
from hanspell import spell_checker

text = "왜 안되냐구 이거 바꿔 외않되"

spelled_text = spell_checker.check(text)
print(spelled_text)

Checked(result=True, original='왜 안되냐구 이거 바꿔 외않되', checked='왜 안되냐고 이거 바꿔 왜 안돼', errors=2, words=OrderedDict([('왜', 0), ('안되냐고', 0), ('이거', 0), ('바꿔', 0), ('안돼', 0)]), time=0.04292798042297363)


In [8]:
df.head()

,brand_name,product_name,write_dt_lst,rating_lst,content_lst
0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행가서 잘 썼습니당~ 유럽용은 안써도 될듯 ㅠ
1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장난거 아니였으면 하네요
3,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,돈 추가해서 구매하는게 더 좋네요 금액 차이 얼마 안나는데 기능은 더 좋아요
4,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,너무 좋아요 해외여행 가면 두루두루 쓸수 있겠어요
5,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,찾던아이 잘 샀어용 번창하세요


In [9]:
from hanspell import spell_checker
import xml.etree.ElementTree as ET

# review_lst = review_df_1['spacing_review'].to_list()

spelled_checked_text = []
for lst in df['content_lst']:
    try:
        result = spell_checker.check(lst)
        spelled_text = result.checked
    except ET.ParseError:
        spelled_text = lst  # 오류 발생 시 원래 텍스트 사용
    except Exception as e:
        spelled_text = lst  # 기타 예외 발생 시 원래 텍스트 사용
        print(f"Error processing text: {lst}\nException: {e}")
    
    spelled_checked_text.append(spelled_text)

#df['spelled_checked_text'] = spelled_checked_text
df['content_lst'] = spelled_checked_text

In [10]:
def upper_text(text):
    if pd.notnull(text):  # 텍스트가 Null이 아닌 경우에만 변환
        return text.upper()
    return text  # Null 값은 그대로 반환

#df['spelled_checked_text'] = df['spelled_checked_text'].apply(upper_text)
df['content_lst'] = df['content_lst'].apply(upper_text)

In [11]:
def replace_text(text):
  text = str(text)
  text = text.replace('쌈ㅎ','저렴해요')
  text = text.replace('안되는...','안됩니다')
  text = text.replace('ㅎ','')
  text = text.replace('ㅋ','')
  text = text.replace('루♡','')
  text = text.replace('♡','')
  text = text.replace('굳','굿')
  text = text.replace('ㅜ','')
  text = text.replace('중입니다)','중 입니다')
  text = text.replace('하나에요','하나예요')
  text = text.replace('있었어요','있어요')
  text = text.replace('욥','요')
  text = text.replace('오겠씀다~~ㅎㅎ/','오겠습니다')
  text = text.replace('후에~','')
  text = text.replace('개용','개요')
  text = text.replace('ㅅ','')
  text = text.replace('오ㅡ','')
  text = text.replace('여행가고싶어지네요ㅋㅋ','여행 가고 싶어지네요')
  text = text.replace('좋습니다!!!!','좋습니다')
  text = text.replace('구매확정해요','구매 확정해요')
  text = text.replace('완죤빨라요ㅋㅋ','완전 빨라요')
  text = text.replace('ㅡ','')
  text = text.replace('귀요미에요','귀여워요')
  text = text.replace('최고여요','최고예요')
  text = text.replace('사랑하여','사랑해요')
  text = text.replace('배송되었습니다','배송 되었습니다')
  text = text.replace('좋아요ㅔ','좋아요')
  text = text.replace('아니에요.','아니예요')
  text = text.replace('구매확정했습니다','구매 확정했습니다')
  text = text.replace('bbb','굿')
  text = text.replace('쓸ㄲ너ㅣㅇ르','')
  text = text.replace('같고요','같아요')
  text = text.replace('같네요','같아요')
  text = text.replace('씁니다','습니다')
  text = text.replace('넘좋와여','너무 좋아요')
  text = text.replace('굿입니다','굿')
  text = text.replace('목배게최고에요','목배게 최고예요')
  text = text.replace('감동이에요','감동이예요')
  text = text.replace('아이디어템','아이디어 탬')
  text = text.replace(':D','')
  text = text.replace('감사핮니다','감사합니다')
  text = text.replace('템','탬')
  text = text.replace('든든 합니다','든든합니다')
  text = text.replace('받아보았어요','받아봤어요')
  text = text.replace('가타용','같아요')
  text = text.replace('바랍니아','바랍니다')
  text = text.replace('사업번창하세요','사업 번창하세요')
  text = text.replace('쓸겠습니다','쓰겠습니다')
  text = text.replace('구매완료입니다','구매완료')
  text = text.replace('짱빠름','짱 빠름')
  text = text.replace('됩니닽','됩니다')
  text = text.replace('닙니다ㅛ','닙니다')
  text = text.replace('잘한듯해요','잘한듯')
  text = text.replace('가세여','가세요')
  text = text.replace('잘샀다템','잘샀어요')
  text = text.replace('좋아요좋아요좋오ㅓㅗㅇ좋아요','좋아요 좋아요')
  text = text.replace('써봐겠ㅆ네요','써봐야겠네요')
  text = text.replace('만족이에요','만족이예요')
  text = text.replace('아까워요.','아까워요')
  text = text.replace('ajjqjahaja','')
  text = text.replace('잚슬겡ㅎ','잘쓸게요')
  text = text.replace('잘했아요','잘했어요')
  text = text.replace('괜찬ㅍ아요','괜찮아요')
  text = text.replace('예쁨','예뻐요')
  text = text.replace('편하쥬','편해요')
  text = text.replace('감사함다','감사합니다')
  text = text.replace('핳핳','')
  text = text.replace('굿입니다','굿')
  text = text.replace('으흐흐','')
  text = text.replace('빨라요ㅗ','빨라요')
  text = text.replace('이븜','이쁨')
  text = text.replace('ㄹㅇ','')
  text = text.replace('되매요','되네요')
  text = text.replace('믿어보겠음다','믿어보겠습니다')
  text = text.replace('아니에요','아니예요')
  text = text.replace('만족합니다.ㅣ','만족합니다')
  text = text.replace('되고요','됩니다')
  text = text.replace('딱입니딘','딱입니다')
  text = text.replace('기대되어요','기대됩니다')
  text = text.replace('왔습디다','왔습니다')
  text = text.replace('맀어용','랐어요')
  text = text.replace('강강히','')
  text = text.replace('만족이여','만족이요')
  text = text.replace('사용했니다.','사용했습니다')
  text = text.replace('충전빨리되구죻아요','충전 빨리되고 좋아요')
  text = text.replace('잘됩니다잉','잘됩니다')
  text = text.replace('사용했답니다','사용했어요')
  text = text.replace('가타요','같아요')
  text = text.replace('최곱니다','최고입니다')
  text = text.replace('빠르네요','빠릅니다')
  text = text.replace('기릉','기를')
  text = text.replace('받았습니아','받았습니다')
  text = text.replace('ok.','ok')
  text = text.replace('가져감','가져갑니다')
  text = text.replace('익니다','입니다')
  text = text.replace('수용되오ㅡ','수용되요')
  text = text.replace('맨죡','만족')
  text = text.replace('ㄹ','')
  text = text.replace('구매예정이에요','구매 예정이예요')
  text = text.replace('좋은듓','좋은듯')
  text = text.replace('매우저롐하게잘구매했어요감사하네여','매우 저렴하게 잘 구매했어요 감사해요')
  text = text.replace('만적합니다','만족합니다')
  text = text.replace('ㅈ번창하세요','번창하세요')
  text = text.replace('전이에요','전이예요')
  text = text.replace('조어요','좋아요')
  text = text.replace('젛아요','좋아요')
  text = text.replace('됩니더','됩니다')
  text = text.replace('굿입디다','굿')
  text = text.replace('갘아어요','')
  text = text.replace('ㄷ,','')
  text = text.replace('잘되겠쥬','잘되겠죠')
  text = text.replace('쓰게습니도','쓰겠습니다')
  text = text.replace('하고요','하고')
  text = text.replace('저뇽가방도있네요','전용가방도 있네요')
  text = text.replace('sjsjsjjsjs','')
  text = text.replace('좋겠구먼요','좋겠네요')
  text = text.replace('좋아욫','좋아요')
  text = text.replace('반퓸합니다','반품합니다')
  text = text.replace('꿋꿋입니다','굿')
  text = text.replace('잘씅게용','잘쓸게요')
  text = text.replace('땡큐네요','감사합니다')
  text = text.replace('베릐굿','굿')
  text = text.replace('효오','')
  text = text.replace('☆','')
  text = text.replace('ᄒ','')
  text = text.replace('썼어ᄉ','썼어요')
  text = text.replace('사용시뉴용할듯욕','사용시 유용할 듯')
  text = text.replace('있아요','있어요')
  text = text.replace('아다아다아','')
  text = text.replace('아다 아다 아','')
  text = text.replace('감사합니다아','감사합니다')
  text = text.replace('goood','굿')
  text = text.replace('싱픔굿배소오쟈웁','상품 굿 배송')
  text = text.replace('쓸겡','쓸게요')
  text = text.replace('굿국국숙','굿굿굿')
  text = text.replace('졸아요','좋아요')
  text = text.replace('USC','USB')
  text = text.replace('A S','AS')
  text = text.replace(' ㅐ외','해외')
  text = text.replace('ᄆ','')
  text = text.replace('ᅳᄋ ᄋ','')
  text = text.replace('배슝','배송')
  text = text.replace('ᅲ','')
  text = text.replace('쓸ᄁ너ᅵᄋ르','쓸게요')
  text = text.replace(' ♥','')
  text = text.replace('ᄏ','')
  text = text.replace('강 추','강추')
  text = text.replace('★','')
  text = text.replace('■','')
  
  # text = text.replace('USB', '유에스비')
  text = text.replace('필수 템', '필수템')
  text = text.replace('멀티 어댑터', '멀티어댑터')
  text = text.replace('EU 타입', 'EU타입')
  text = text.replace('느리편', '느린편')
  text = text.replace('요거', '이거')
  text = text.replace('여러 개', '여러개') 
  text = text.replace('내구 성', '내구성')
  text = text.replace('걱정 없이', '걱정없이')
  text = text.replace('부담 없는', '부담없는')
  text = text.replace('내구 성', '내구성')
  text = text.replace('선망', '선만')
  text = text.replace('필요 없음', '필요없음')
  text = text.replace('여향', '여행')
  text = text.replace('있렁서', '있어서')
  text = text.replace('필요 없음', '필요없음')
  text = text.replace('봏아요', '좋아요')
  text = text.replace('죠아용', '좋아요')
  text = text.replace('필요 없음', '필요없음')
  text = text.replace('좋았늡', '좋았습')
  text = text.replace('개벼운', '가벼운')
  text = text.replace('티스트', '테스트')

  text = text.replace('보관 백', '보관백')
  text = text.replace('TYPE', '타입')
  text = text.replace('C TO C', 'C타입')
  text = text.replace('CTOC', 'C타입')
  text = text.replace('C2C', 'C타입')
  text = text.replace('USB-C', 'USB포트 C타입')
  text = text.replace('USB-A', 'USB포트 A타입')
  text = text.replace('C TYPE', 'C타입')
  text = text.replace('C 타입','C타입')
  text = text.replace('씨타입','C타입')
  text = text.replace('씨 타입', 'C타입')
  text = text.replace('A 타입','A타입')
  text = text.replace('에이타입','A타입')
  text = text.replace('에이 타입', 'A타입')
  text = text.replace('EU 타입', 'EU타입')
  text = text.replace('PD 충전','PD충전')
  text = text.replace('PD 잭', 'PD잭')
  text = text.replace('급 속','급속')
  text = text.replace('포드','포트')
  text = text.replace('포드만','포트')
  text = text.replace('포드리','포트')
  text = text.replace('핸폰','핸드폰')
  text = text.replace('휴대폰', '핸드폰')
  text = text.replace('휴대전화', '핸드폰')
  text = text.replace('11볼트', '110V')
  text = text.replace('22볼트', '220V')
  text = text.replace('11V', '110V')
  text = text.replace('22V', '220V')
  # text = text.replace('35W', '35와트')
  # text = text.replace('45W', '45와트')
  # text = text.replace('65W', '65와트')
  text = text.replace('포터', '포트')
  # text = text.replace('USB', 'USB포트') 
  text = text.replace('USB 포트', 'USB포트')
  text = text.replace('동시 충전', '동시충전')
  text = text.replace('고속 충전', '고속충전')
  text = text.replace('동시 이용', '동시이용')
  text = text.replace('동시 사용', '동시사용')
  text = text.replace('여러 나라', '여러나라')
  text = text.replace('코넥터', '커넥터')
  # 1개짜리는 제외
  return text

In [12]:
df['content_lst'] = df['content_lst'].apply(replace_text)

In [13]:
df.head()

,brand_name,product_name,write_dt_lst,rating_lst,content_lst
0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ
1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요
3,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,돈 추가해서 구매하는 게 더 좋네요 금액 차이 얼마 안 나는데 기능은 더 좋아요
4,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,너무 좋아요 해외여행 가면 두루두루 쓸 수 있겠어요
5,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,찾던 아이 잘 샀어요 번창하세요


kss하고

In [ ]:
# 한국어 문장 분리기
#!pip install kss

In [15]:
import kss
from kss import split_sentences

def kss_sentence_split(text):
    return kss.split_sentences(text)

df['kss_split_text'] = df['content_lst'].apply(kss_sentence_split)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [16]:
df.head()

,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text
0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,"[여행 가서 잘 썼습니다~, 유럽용은 안 써도 될 듯 ㅠ]"
1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,"[여행에서 사용하려고요, 고장 난 거 아니었으면 하네요]"
3,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,돈 추가해서 구매하는 게 더 좋네요 금액 차이 얼마 안 나는데 기능은 더 좋아요,"[돈 추가해서 구매하는 게 더 좋네요, 금액 차이 얼마 안 나는데 기능은 더 좋아요]"
4,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,너무 좋아요 해외여행 가면 두루두루 쓸 수 있겠어요,"[너무 좋아요, 해외여행 가면 두루두루 쓸 수 있겠어요]"
5,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,찾던 아이 잘 샀어요 번창하세요,"[찾던 아이 잘 샀어요, 번창하세요]"


In [18]:
# 여러 단어 기준으로 문자열 분리하는 함수 정의
def split_by_multiple_words(text, words):
    parts = [text]
    for word in words:
        new_parts = []
        for part in parts:
            new_parts.extend(split_by_word(part, word))
        parts = new_parts
    return parts

# 단어 기준으로 문자열 분리하는 기본 함수 정의
def split_by_word(text, word):
    parts = []
    start = 0
    while True:
        index = text.find(word, start)
        if index == -1:
            break
        parts.append(text[start:index + len(word)])
        start = index + len(word)
    parts.append(text[start:])
    return parts

# 여러 단어 기준으로 분리할 단어 리스트 정의
words_to_split = ['빠르고', '스럽고', ',']

# split_text 열을 분리하여 새로운 열 생성
df2 = df.explode(['kss_split_text'])
#df2['all_split_text'] = df2['kss_split_text'].apply(lambda x: split_by_multiple_words(x, words_to_split))
df2['kss_split_text'] = df2['kss_split_text'].apply(lambda x: split_by_multiple_words(x, words_to_split))
df2 = df2.explode(['kss_split_text'])
df2.head(3)

,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text
0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,여행 가서 잘 썼습니다~
0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,유럽용은 안 써도 될 듯 ㅠ
1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,여행에서 사용하려고요


In [19]:
df2.to_csv('C:/Users/songy/Desktop/Data/sentiment_data/travel_adaptor_total.csv',encoding='utf-8-sig')

단순리뷰

In [3]:
df2 = pd.read_csv('C:/Users/songy/Desktop/Data/sentiment_data/travel_adaptor_total.csv')

In [4]:
df2.head(3)

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text
0,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,여행 가서 잘 썼습니다~
1,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,유럽용은 안 써도 될 듯 ㅠ
2,1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,여행에서 사용하려고요


In [5]:
#### 위 함수 강화 버전 (고유 단어 개수 기반 추가 필터링)

def text_again_dev(text):
  text = str(text)
  words = text.split()

  # 중복 단어 케이스 (True: 중복 없음)
  result1 = (len(words)==len(set(words)))

  # 연속 단어 케이스
  result2 = 0
  for i in range(1, len(words)):
    if words[i] == words[i-1]:
      result2 = 1
    else:
      result2 = 0

  # 두 케이스가 모두 만족하는 경우 1 반환, 아니면 0 반환 (중복 & 연속)
  result3 = 0
  if (str(result1)=='False') & (result2==1):
    result3 = 1
  else:
    result3 = 0

  #### 세번째 케이스 (set 개수)
  set_len = len(set(words))
  if (result3 == 1) & (set_len <= 3):
    return 1
  else :
    return 0

df2['len'] = df2['content_lst'].apply(lambda x : len(str(x)))
df2['repeated_final'] = df2['content_lst'].apply(text_again_dev)
df2.head(3)

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text,len,repeated_final
0,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,여행 가서 잘 썼습니다~,29,0
1,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,유럽용은 안 써도 될 듯 ㅠ,29,0
2,1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,여행에서 사용하려고요,28,0


In [6]:
# 12자 이하 단순처리 
# 12자 초과이면서 repeated final 1인것 단순 처리
# 나머지 일반


In [7]:
# 단순 / 일반 리뷰 처리
df2['review_type'] = df2.apply(lambda x: "단순" if (x['len']<=12) | ((x['len']>12) & (x['repeated_final']==1)) else "일반", axis=1)
df2.head(3)

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text,len,repeated_final,review_type
0,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,여행 가서 잘 썼습니다~,29,0,일반
1,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,유럽용은 안 써도 될 듯 ㅠ,29,0,일반
2,1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,여행에서 사용하려고요,28,0,일반


In [8]:
round((df2['review_type'].value_counts(normalize=True))*100,2)

review_type
일반    92.82
단순     7.18
Name: proportion, dtype: float64

In [9]:
df2.to_csv('C:/Users/songy/Desktop/Data/sentiment_data/travel_adaptor_review_type.csv', encoding='utf-8-sig')

In [10]:
df3 = pd.read_csv('C:/Users/songy/Desktop/Data/sentiment_data/travel_adaptor_review_type.csv', index_col=0)
df3.head(3)

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text,len,repeated_final,review_type
0,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,여행 가서 잘 썼습니다~,29,0,일반
1,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,유럽용은 안 써도 될 듯 ㅠ,29,0,일반
2,1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,여행에서 사용하려고요,28,0,일반


2차 처리

In [11]:
keywords = ['잘 사용했습니다', '잘 이용했습니다', '잘 사용했어요','잘 썼습니다', '잘 썼어요', '잘 쓰고왔어요', '잘 받았습니다', '잘 받았어요', '사용하려고요', '잘 사용', '많이 파세요', '아주 좋아요',
            '잘 이용하고 있어요', '잘 이용', '잘 쓰고있습니다', '잘 쓰고 있습니다', '잘 쓰고있어요', '잘 쓰겠슴당', '잘 되네요', '잘 작동돼요','잘 됩니다', '잘 이용하겠습니다', '잘 사용하겠습니다', '잘 쓰겠습니다', '잘 샀어요', '잘 써볼게요', '잘 쓸게요',
            '구매했는데','아직', '여행', '사용 전', '번창','최고예요','좋습니다','강추합니다', '여행 가려고', '여행 가서',  '잘 돼요', '잘 산 것 같아요', '잘 쓰고 있어요', '그냥', '써볼게요',
            '구매 했어요','구매 했습니다', '너무 좋아요', '좋습니다', '너무 좋습니다', '않았습니다', '않았네요', '빨리','후기','구입','주문','리뷰','수령','협찬','감사합니다','좋았습니다','좋네요', '좋아요', '좋을 거', '좋을 것', '면세점', '좋아 보여요', '좋겠네요',
            '수고하세요','만족합니다','사용 후 후기', '좋아하세요','좋겠어요', '안좋아요', '잘됩니다', '귀엽네요', '기대됩니다', '바랍니다', '훌륭합니다', '훌륭하네요', '유용하네요', '유용합니다', '아쉬웠네요',
            '만족스럽습니다', '잘 되네요', '이거 사세요', '강추입니다', '추천', '조만간', '편리', '이용', '잘', '좋은 거', '작동', '신행', '신혼여행', '신혼 여행', '좋은 것', '좋은 거','편리', '좋아하네요', '문제없었어요',
            '좋아 보입니다', '마음', '맘', '좋더라고요', '편했습니다', '최고', '와우', '대만족', '필수', '여행 필수품', '튼튼', '매력',
            '굿', '굳', 'N번', 'N년', '애용', '항상', '늘', '꾸준히', '쭉', '애용', '항상', '늘', '매번', '매일', '계속', '몇 년', '조타' ,'조아']

common_entity = ['배송','프로모션', '가격', '가성비', '싼값', '구매', '재구매','포장','판매','저렴','할인','촉진','AS', '문의', '답변','상담'] #재구매

entity = ['중량','색상', '색깔', '색', '디자인', '무게','사이즈','크기','성능','속도','퀄리티','모양','수명','안전성','휴대','콤팩트','만능',
          '사용성','호환', '품질','스펙','견고','영구','기능','구성','인증', '소재']   # 소재

keywords_patterns = '|'.join(keywords)
common_entity_patterns = '|'.join(common_entity)
entity_patterns = '|'.join(entity)
# nation_patterns = '|'.join(nation)

def clean_text(text):
    if isinstance(text, str):
        return re.sub(r'[-=+,#/\?:^$.@*"※~&%ㆍ!』\\‘·|\(\)\[\]\<\>`\'…\"\“’]', '', text)
    else:
        return ""
    
df3['kss_split_text'].apply(clean_text)

#### 단순으로 추가 

df3.loc[(df3['repeated_final']==0) & 
        (df3['content_lst'].str.contains(keywords_patterns, na=False)) &
        (~df3['content_lst'].str.contains(common_entity_patterns, na=False)) &
        (~df3['content_lst'].str.contains(entity_patterns, na=False)) &
        (df3['content_lst'].str.len()<=28),"review_type"]="단순"

df3.loc[(df3['repeated_final']==0) & 
        (df3['kss_split_text'].str.contains(keywords_patterns, na=False)) &
        (~df3['kss_split_text'].str.contains(common_entity_patterns, na=False)) &
        (~df3['kss_split_text'].str.contains(entity_patterns, na=False)),"review_type"]="단순"

df3.loc[(df3['repeated_final']==0) & 
        (~df3['kss_split_text'].str.contains(common_entity_patterns, na=False)) &
        (~df3['kss_split_text'].str.contains(entity_patterns, na=False)) &
        (df3['kss_split_text'].str.len()<=7),"review_type"]="단순"

df3.head(3)

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text,len,repeated_final,review_type
0,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,여행 가서 잘 썼습니다~,29,0,단순
1,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,유럽용은 안 써도 될 듯 ㅠ,29,0,일반
2,1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,여행에서 사용하려고요,28,0,단순


In [12]:
df3['review_type'].value_counts()

review_type
단순    15299
일반     9229
Name: count, dtype: int64

서비스 / 제품 구분

In [13]:
df3['detail_review_type'] = ''

In [14]:
df3.loc[(df3['content_lst'].str.contains(common_entity_patterns, na=False)) &
        (~df3['content_lst'].str.contains(entity_patterns, na=False)),"detail_review_type"]="service"

In [15]:
df3.loc[~((df3['content_lst'].str.contains(common_entity_patterns, na=False)) &
        (~df3['content_lst'].str.contains(entity_patterns, na=False))),"detail_review_type"]="product"

In [16]:
df3['detail_review_type'].value_counts()

detail_review_type
product    17802
service     6726
Name: count, dtype: int64

In [17]:
df3[df3['detail_review_type']=='service'].head()

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text,len,repeated_final,review_type,detail_review_type
13,8,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240713,5,배송이 너무 빠릅니다. 좋은 제품 저렴하게 구입 잘했습니다,배송이 너무 빠릅니다.,32,0,일반,service
14,8,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240713,5,배송이 너무 빠릅니다. 좋은 제품 저렴하게 구입 잘했습니다,좋은 제품 저렴하게 구입 잘했습니다,32,0,일반,service
17,10,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240713,3,북유럽 여행 예정이라 구매했어요 잘 됐으면 좋겠어요,북유럽 여행 예정이라 구매했어요,28,0,일반,service
18,10,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240713,3,북유럽 여행 예정이라 구매했어요 잘 됐으면 좋겠어요,잘 됐으면 좋겠어요,28,0,단순,service
19,11,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240711,5,가성비 좋고 제품도 깔끔하네요... 감사합니다.,가성비 좋고 제품도 깔끔하네요...,26,0,일반,service


In [18]:
df3[df3['detail_review_type']=='product'].head()

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,kss_split_text,len,repeated_final,review_type,detail_review_type
0,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,여행 가서 잘 썼습니다~,29,0,단순,product
1,0,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240717,5,여행 가서 잘 썼습니다~ 유럽용은 안 써도 될 듯 ㅠ,유럽용은 안 써도 될 듯 ㅠ,29,0,일반,product
2,1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,여행에서 사용하려고요,28,0,단순,product
3,1,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,4,여행에서 사용하려고요 고장 난 거 아니었으면 하네요,고장 난 거 아니었으면 하네요,28,0,단순,product
4,3,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,돈 추가해서 구매하는 게 더 좋네요 금액 차이 얼마 안 나는데 기능은 더 좋아요,돈 추가해서 구매하는 게 더 좋네요,44,0,일반,product


In [19]:
df3.to_csv('C:/Users/songy/Desktop/Data/sentiment_data/travel_adaptor_detail_type.csv',encoding='utf-8-sig')